In [169]:

import pandas as pd
import numpy as np
from pathlib import Path
from IPython.display import display, HTML
import plotly.express as px
import dataframe_image as dfi
pd.options.mode.chained_assignment = 'warn'  # default=
pd.options.plotting.backend = "plotly"
PATH_CACHE = Path("../../data/algos_results.csv").resolve()
print(PATH_CACHE)
pd.set_option('display.max_columns', None)  

C:\Users\robin\Documents\Cours\Poly\Recherche\MaitriseClustering\data\algos_results.csv


In [176]:
df = pd.read_csv(PATH_CACHE)
init_type = "kmean+"
local_df = df[df["init_type"] == init_type].copy()
local_df.loc[:,"algorithm"] = local_df["ImprClass"]+"_" + local_df["iteration_order"]
local_df.loc[:,"index"] = local_df["uuid_points_coords"]+"_" + local_df["uuid_points_assign"]+"_"+local_df["num_points"].astype(str)
local_df = local_df.drop(["ImprClass","init_type","iteration_order","uuid_points_coords","uuid_points_assign","time","num_iter"],axis=1)
Lcluster_concat = []
for num_clusters in [4,8]:
    Linit_concat = []
    df_clusters = local_df[local_df["num_clusters"] == num_clusters].copy()
    for algorithm in pd.unique(df_clusters["algorithm"]):
        tmp_df = df_clusters[df_clusters["algorithm"] == algorithm].copy()
        tmp_df.set_index(["index"],inplace=True)
        if "Best" in algorithm:
            backup_tmp_df = tmp_df.copy()
        else:
            tmp_df.loc[:,"cost_augm"] = tmp_df["end_cost"].copy() - backup_tmp_df["end_cost"].copy()
        tmp_df = tmp_df.groupby("num_points",as_index=False).mean()
        tmp_df = tmp_df.drop(['num_clusters'],axis=1)
        tmp_df.set_index(["num_points"],inplace=True)
        Linit_concat.append(tmp_df)
    Lcluster_concat.append(pd.concat(Linit_concat,axis=1, keys=["BI_BACK", "FI_CURR", "FI_BACK", "FI_RANDOM"]))
df100_cluster = pd.concat(Lcluster_concat, axis=1, keys=["4 clusters", "8 clusters","16 clusters", "64 clusters"])
df100_cluster

4 clusters                                            \
                 BI_BACK                     FI_CURR                 
               init_cost      end_cost     init_cost      end_cost   
num_points                                                           
20           8983.228952   5666.104084   8983.228952   5708.917922   
30          15947.727810   9191.637699  15947.727810   9271.308567   
40          24062.430269  14652.902367  24062.430269  14647.878613   
50          30751.879945  18844.731802  30751.879945  18695.749330   
60          36319.875562  22199.490576  36319.875562  22162.996923   
70          44098.577466  28808.300117  44098.577466  28414.490180   
80          49739.737326  32074.180727  49739.737326  31827.106213   
90          58292.689820  37262.165567  58292.689820  37148.989520   
100         65484.981246  42090.919486  65484.981246  42089.270929   

                                                                              \
                             FI_BACK                               FI_RANDOM   
             cost_augm     init_cost      end_cost   cost_augm     init_cost   
num_points                                                                     
20           42.813838   8983.228952   5713.990093   47.886009   8983.228952   
30           79.670867  15947.727810   9271.308565   79.670866  15947.727810   
40           -5.023754  24062.430269  14656.341074    3.438707  24062.430269   
50         -148.982471  30751.879945  18702.612748 -142.119054  30751.879945   
60          -36.493653  36319.875562  22161.020119  -38.470457  36319.875562   
70         -393.809937  44098.577466  28414.509838 -393.790279  44098.577466   
80         -247.074513  49739.737326  31815.489026 -258.691701  49739.737326   
90         -113.176047  58292.689820  37148.989499 -113.176068  58292.689820   
100          -1.648557  65484.981246  42073.397762  -17.521724  65484.981246   

                                        8 clusters                \
                                           BI_BACK                 
                end_cost   cost_augm     init_cost      end_cost   
num_points                                                         
20           5716.541750   50.437666   3489.819105   1845.740767   
30           9271.308566   79.670867   7606.568303   3989.666723   
40          14665.506216   12.603849  12658.359175   6233.574676   
50          18687.750081 -156.981721  17286.611220   8700.293555   
60          22167.579978  -31.910598  21571.692702  10927.631942   
70          28403.214887 -405.085230  26887.838907  12657.740678   
80          31815.851886 -258.328840  30635.349981  14490.125038   
90          37148.989648 -113.175920  36406.312258  16914.089003   
100         42073.397691  -17.521795  41184.420297  19395.483381   

                                                                  \
                 FI_CURR                                 FI_BACK   
               init_cost      end_cost   cost_augm     init_cost   
num_points                                                         
20           3489.819105   1905.375321   59.634554   3489.819105   
30           7606.568303   4000.296804   10.630082   7606.568303   
40          12658.359175   6178.820095  -54.754581  12658.359175   
50          17286.611220   8561.910829 -138.382726  17286.611220   
60          21571.692702  10431.772171 -495.859772  21571.692702   
70          26887.838907  12524.753321 -132.987357  26887.838907   
80          30635.349981  14255.325561 -234.799478  30635.349981   
90          36406.312258  16886.473945  -27.615058  36406.312258   
100         41184.420297  18872.270076 -523.213305  41184.420297   

                                                                              
                                         FI_RANDOM                            
                end_cost   cost_augm     init_cost      end_cost   cost_augm  
num_points                                             

In [177]:
dfi.export(df100_cluster,"mytable.png")

In [167]:
class SignColorFormatter:
    def format(self,df):
        original_df = df.copy()
        df.loc[
            original_df["cost_augm_BI_ref"] < 0, "cost_augm_BI_ref"
        ] = "decrease"
        df.loc[
            original_df["cost_augm_BI_ref"] > 0, "cost_augm_BI_ref"
        ] = "increase"
        df.loc[
            original_df["cost_augm_BI_ref"] == 0, "cost_augm_BI_ref"
        ] = "no_change"
        return df
    def get_colors(self):
        return {"decrease":"red","increase":"green","no_change":"blue"}
class MultipleRanges:
    def __init__(self):
        self.dico = {"#9E2A2B":[100,99999],
                     "#F3722C":[10,100],
                     "#F8961E":[1,10],
                     "#F9844A":[0,1],
                     "white": 0,
                     "#C6FF94":[-1,0],
                     "#00BC78":[-10,-1],
                     "#4D908E":[-100,-10],
                     "#245F84":[-99999,-100],
                     }
    def format(self,df):
        original_df = df.copy()
        for ranges in self.dico.values():
            if isinstance(ranges,list):
                if  ranges[1] in self.dico.values():
                    localization = (original_df["cost_augm_BI_ref"] > ranges[0]) & (original_df["cost_augm_BI_ref"] < ranges[1])
                else:
                    localization = (original_df["cost_augm_BI_ref"] > ranges[0]) & (original_df["cost_augm_BI_ref"] <= ranges[1])
                    
                df.loc[
                    localization, "cost_augm_BI_ref"
                ] = str(ranges)
            elif isinstance(ranges,int):
                df.loc[
                    original_df["cost_augm_BI_ref"] == ranges, "cost_augm_BI_ref"
                ] = str(ranges)
            else:
                print(f"WARNING: ranges {ranges} is not a list or int")
        return df
    def get_colors(self):
        return {str(ranges):color for color,ranges in self.dico.items()}
    
def reformat(df, init="random", average=True, color_formatter=None):
    tmp_df = df[df["init_type"] == init].copy()
    tmp_df.loc[:, "algorithm"] = tmp_df["ImprClass"] + "_" + tmp_df["iteration_order"]
    tmp_df.loc[:, "index"] = (
        tmp_df["uuid_points_coords"]
        + "_"
        + tmp_df["uuid_points_assign"]
        + "_"
        + tmp_df["num_points"].astype(str)
    )
    tmp_df = tmp_df.drop(
        [
            "init_cost",
            "ImprClass",
            "init_type",
            "iteration_order",
            "uuid_points_coords",
            "uuid_points_assign",
        ],
        axis=1,
    )
    Lnew = []
    for num_clust in [2, 4, 8, 16, 32, 64, 128]:
        df_tmp = tmp_df[tmp_df["num_clusters"] == num_clust].copy()
        Ldf = []
        for algorithm in pd.unique(df_tmp["algorithm"]):
            df_tmp1 = df_tmp[df_tmp["algorithm"] == algorithm].copy()
            df_tmp1.set_index(["index"], inplace=True)
            if "Best" in algorithm:
                backup_tmp_df = df_tmp1.copy()
                backup_tmp_df.loc[:, "algorithm"] = algorithm
                backup_tmp_df.loc[:, "num_clusters"] = num_clust
                continue
            else:
                current = df_tmp1
                df_tmp1 = df_tmp1.assign(
                    cost_augm_BI_ref=current["end_cost"].copy()
                    - backup_tmp_df["end_cost"].copy()
                )
            if average:
                df_tmp1 = df_tmp1.groupby("num_points", as_index=False).mean()
            df_tmp1.loc[:, "algorithm"] = algorithm
            df_tmp1.loc[:, "num_clusters"] = num_clust
            new_df_tmp1 = df_tmp1.copy()
            if color_formatter is not None:
                new_df_tmp1 = color_formatter.format(new_df_tmp1)
            Ldf.append(new_df_tmp1)
        if len(Ldf) > 0:
            Lnew.append(pd.concat(Ldf, axis=0))
    new_df = pd.concat(Lnew, axis=0)
    return new_df


def table(init_df, init="random"):
    df = reformat(init_df.copy(), init, average=False)
    return display(df)


def plot(init_df, init="random", color_formatter=None):
    df = reformat(init_df.copy(), init, color_formatter=color_formatter)
    # fig = px.scatter_3d(
    #     df,
    #     x="num_points",
    #     y="algorithm",
    #     z="num_clusters",
    #     log_z=True,
    #     color="cost_augm_BI_ref",
    #     color_discrete_map=color_formatter.get_colors() if color_formatter is not None else None,
    #     template="plotly_dark",
    #     width=1000,
    #     height=800,
    #     labels={
    #                  "num_points": "Nombre de points",
    #                  "algorithm": "Algorithme",
    #                  "num_clusters": "Nombre de clusters",
    #                  "cost_augm_BI_ref": "Augmentation du coût<br> par rapport à<br> Best Improvement"
    #              },
    #     category_orders={"cost_augm_BI_ref": (color_formatter.get_colors().keys()) } if color_formatter is not None else None ,
    #     title=f"Augmentation du coûtpar rapport à Best Improvement (CURR)<br>avec initialisation {init}",
    # )
    fig = px.scatter(
        df[df["algorithm"] == "CallbackFirstImprovement_CURR"],
        x="num_points",
        y="num_clusters",
        log_y=True,
        color="cost_augm_BI_ref",
        color_discrete_map=color_formatter.get_colors() if color_formatter is not None else None,
        width=1000,
        height=800,
        labels={
                     "num_points": "Nombre de points",
                     "algorithm": "Algorithme",
                     "num_clusters": "Nombre de clusters",
                     "cost_augm_BI_ref": "Augmentation du coût<br> par rapport à<br> Best Improvement"
                 },
        category_orders={"cost_augm_BI_ref": (color_formatter.get_colors().keys()) } if color_formatter is not None else None ,
        title=f"Augmentation du coût de FirstImprovement_CURR par rapport à Best Improvement (CURR)<br>avec initialisation \"{init}\"",
    )
    fig.update_traces(marker=dict(size=12),
                  selector=dict(mode='markers'))
    fig.show()  # "png"


In [168]:
plot(df,"kmean+",color_formatter=MultipleRanges())

In [142]:
plot(df,"random",color_formatter=MultipleRanges())